Mengisi nilai variabel KAGGLE_USERNAME dan KAGGLE KEY agar dapat mendownload dataset langsung ke colab

In [1]:
import os
os.environ['KAGGLE_USERNAME'] = 'arfalrazya'
os.environ['KAGGLE_KEY'] = '4ec402e83cc87e2b5a7e2e23d82c40f2'

Mendownload datasets dengan API yang telah disediakan di halaman dataset

In [2]:
!kaggle datasets download -d hernan4444/anime-recommendation-database-2020

 99% 652M/661M [00:19<00:00, 49.9MB/s]
100% 661M/661M [00:19<00:00, 35.1MB/s]


Melakukan Ekstraksi datasets dengan command unzip

In [3]:
!unzip 'anime-recommendation-database-2020.zip' -d /content/

Archive:  anime-recommendation-database-2020.zip
  inflating: /content/anime.csv      
  inflating: /content/anime_with_synopsis.csv  
  inflating: /content/animelist.csv  
  inflating: /content/html folder/html/1/details.html  
  inflating: /content/html folder/html/1/pictures.html  
  inflating: /content/html folder/html/1/recomendations.html  
  inflating: /content/html folder/html/1/reviews_1.html  
  inflating: /content/html folder/html/1/reviews_10.html  
  inflating: /content/html folder/html/1/reviews_11.html  
  inflating: /content/html folder/html/1/reviews_12.html  
  inflating: /content/html folder/html/1/reviews_13.html  
  inflating: /content/html folder/html/1/reviews_14.html  
  inflating: /content/html folder/html/1/reviews_15.html  
  inflating: /content/html folder/html/1/reviews_16.html  
  inflating: /content/html folder/html/1/reviews_17.html  
  inflating: /content/html folder/html/1/reviews_18.html  
  inflating: /content/html folder/html/1/reviews_19.html  
  i

#Data Loading
Membaca dataset pada file **anime_with_synopsis.csv**. Ini dilakukan menggunakan fungsi **read_csv** dari library pandas





In [4]:
import pandas as pd
 
anime_synopsis = pd.read_csv('anime_with_synopsis.csv')

#Data Understandings

## Melihat informasi dari datasets

Melihat informasi kolom pada dataset anime_synopsis dengan fungsi **info()**. Hasil dari fungsi info memberikan infromasi bahwa terdapat satu tipe data integer pada kolom MAL_ID dan tipe data object untuk kolom lainnya.

In [5]:
anime_synopsis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16214 entries, 0 to 16213
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   MAL_ID     16214 non-null  int64 
 1   Name       16214 non-null  object
 2   Score      16214 non-null  object
 3   Genres     16214 non-null  object
 4   sypnopsis  16206 non-null  object
dtypes: int64(1), object(4)
memory usage: 633.5+ KB


## Melihat Baris pada datasets

Melihat 5 baris teratas dari datasets anime_synopsis. dengan fungsi **head()**. Fungsi tersebut akan menghasilkan 5 baris teratas dari datasets anime_synopsis beserta nilai pada setiap kolom pada baris tersebut.

In [6]:
anime_synopsis.head()

,MAL_ID,Name,Score,Genres,sypnopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...


#Data Preparation
##Pengecekan Missing Value

Melakukan pengecekan untuk missing value menggunakan fungsi **isnull()**, kemudian menjumlhkan nilai pada kolom yang  mengandung missing value. Dari Hasil pengecekan pada datasets **anime_synopsis** terdapat 8 missing value pada kolom synopsis.

In [7]:
anime_synopsis.isnull().sum()

MAL_ID       0
Name         0
Score        0
Genres       0
sypnopsis    8
dtype: int64

## *Missing Value Handling*

Mengganti missing value pada kolom sinopsis dengan nilai string kosong. Hai ini dilakukan agar setiap baris pada kolom synopsis nantinya dapat direpresentasikan dengan TF-IDF. Ini dilakukan dengan fungsi **fillna()** yang akan mengisi nilai yang kosong pada kolom synopsis dengan string kosong.

In [8]:
anime_synopsis['sypnopsis'] = anime_synopsis['sypnopsis'].fillna('')

#Model Development dengan Content Based Filtering
## TF-IDF Vectorizer
Melakukan tokenisasi dengan **TfidfVectorizer()** dari library sklearn dengan masukan kolom synopsis dari datasets anime_synopsis. Tokenisasi dilakukan untuk menemukan representasi fitur penting dari setiap sinopsis.

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(analyzer='word')
tfidf_matrix = tfidf.fit_transform(anime_synopsis['sypnopsis'])
tfidf_matrix.shape

(16214, 45363)

## Membuat matriks TFIDF
Membuat hasil tf-idf dalam bentuk matriks, menggunakan fungsi **todense()**. Ini dilakukan agar dapat menghitung kesamaan hasil tokenisasi dengan fungsi cosine similarity 

In [10]:
tfidf_matrix.todense()

matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.23540208, 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]])

##Similarity Measures
Menghitung derajat kesamaan (similarity degree) antar sinopsis dengan teknik **cosine similarity**. Menggunakan fungsi cosine_similarity dari library sklearn.Perhitungan **cosine similarity** dataframe tfidf_matrix yang kita peroleh pada tahapan sebelumnya. 

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(tfidf_matrix) 
cosine_sim

## Matriks Hasil perhitungan Cosine Similarity

Melihat matriks kesamaan setiap sinopsis anime dengan menampilkan nama anime dalam 5 sampel kolom (axis = 1) dan 10 sampel baris (axis=0). Dari matriks ini dapat dilihat nilai yang dihasil kan untuk setiap element bernilai 0 sampai 1. Sebagai contoh pada element [0,0] nilai kesamaan anime "Dai Yamato Zero-gou" dengan "Niehime to Kemono no Ou" memiliki nilai kesamaan 0.078287	

In [ ]:
cosine_sim_df = pd.DataFrame(cosine_sim, index=anime_synopsis['Name'], columns=anime_synopsis['Name'])
print('Shape:', cosine_sim_df.shape)
 
# Melihat similarity matrix pada setiap resto
cosine_sim_df.sample(5, axis=1).sample(10, axis=0)

##Pembentukan Fungsi Rekomendasi

Membuat fungsi anime_recommendations dengan beberapa parameter:
  **Anime** : Nama anime(index kemiripan dataframe).
-   **Similarity_data** : Dataframe mengenai similarity yang telah kita definisikan sebelumnya.
-   **items** : Nama dan fitur yang digunakan untuk mendefinisikan kemiripan. Dalam hal ini **Name** yang merujuk pada nama anime dan **synopsis** yang merupakan sinopsis dari nama anime yang diberikan.
-   **k** : Banyak rekomendasi yang ingin diberikan.

Fungsi ini akan menghasilkan keluaran berupa Top N rekomendasi, yang nilai N akan disesuaikan pada parameter k) berdasarkan kesamaan pada matriks similarity. Keluaran yang dihasilkan adalah judul anime, sinopsis dan genre.



In [ ]:
def anime_recommendations(anime, similarity_data=cosine_sim_df, items=anime_synopsis[['Name', 'sypnopsis','Genres']], k=5):

    index = similarity_data.loc[:,anime].to_numpy().argpartition(
        range(-1, -k, -1))
    
    closest = similarity_data.columns[index[-1:-(k+2):-1]]
    
    closest = closest.drop(anime, errors='ignore')
 
    return pd.DataFrame(closest).merge(items).head(k)

#Evaluasi
##Melakukan Pengecekan Data
Hal pertama untuk melakukan evaluasi, disini dilihat terlebih dahulu padakah judul anime yang akan dimasukkan ke fungsi rekomendasi ada dalam datasets. Pengecekan dilakukan dengan bantuan fungsi **eq()**. Dalam kasus ini akan dipastikan anime dengan judul One Punch Man ada dalam datasets.

In [ ]:
anime_synopsis[anime_synopsis.Name.eq('One Punch Man')]

##Uji Coba Fungsi
Pada pengujian ini akan dilakukan untuk mendapatkan **Top 5 Recommendation**.
Judul Anime yang telah diperiksa tadi dimasukkan dalam fungsi anime **anime_recommendations()**. Dapat dilihat hasil dari fungsi tersebut menghasilkan 5 buah rekomendasi sesuai dengan parameter k.

In [ ]:
anime_recommendations('One Punch Man')